# Poisson equation

c.f. https://teamcoil.sp.u-tokai.ac.jp/lectures/EL1/Poisson/index.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import numba

In [ ]:
N = 100
X = 1.0
e0 = 8.85e-12
center = np.array((N // 2, N // 2))
delta = X / N
Conv = 1.0e-6
rho = np.zeros((N, N))

In [ ]:
for i in range(N):
    for j in range(N):
        if np.linalg.norm(center - (i, j))*delta < 0.05:
            rho[i, j] = 1.0e-8

In [ ]:
# Eq. (6)
@numba.jit
def calc_phi_at(i, j, phi: np.ndarray, rho: np.ndarray, e0):
    return 0.25*(rho[i, j]*(delta**2)/e0+phi[i+1, j]+phi[i-1, j]+phi[i, j+1]+phi[i, j-1])

@numba.jit
def main_loop():
    phi = np.zeros((N, N), dtype=numba.float32)
    MaxPhi_list = []
    loop = 0
    MaxPhi = 1.0e-10
    while True:
        if loop%1000 == 0:
            print(loop, MaxPhi)

        MaxErr = CurErr = 0
        for i in range(1, N-1):
            for j in range(1, N-1):
                Prev_phi = phi[i, j]
                phi[i, j] = calc_phi_at(i, j, phi, rho, e0)

                if MaxPhi < abs(phi[i, j]):
                    MaxPhi = phi[i, j]

                CurErr = abs(phi[i, j] - Prev_phi) / MaxPhi

                if MaxErr < CurErr:
                    MaxErr = CurErr
        MaxPhi_list.append(MaxErr)
        loop += 1
        if MaxErr <= Conv:
            return phi, MaxPhi_list

In [ ]:
%%time

phi, MaxPhi_list = main_loop()

In [ ]:
with open('rho.pkl', 'wb') as fout:
    pickle.dump(rho, fout)
with open('phi.pkl', 'wb') as fout:
    pickle.dump(phi, fout)

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
plt.plot(range(len(MaxPhi_list)), MaxPhi_list)
plt.show()

## Visualization of the electrostatic potential

In [ ]:
fig, ax = plt.subplots()
xs, ys = np.meshgrid(np.arange(N), np.arange(N))
zs = phi[xs, ys]
xs_, ys_ = np.meshgrid(np.arange(N)*delta, np.arange(N)*delta)
im = ax.pcolormesh(xs_, ys_, zs, vmin=np.min(phi), vmax=np.max(phi), cmap='rainbow') # or jet
ax.contour(xs_, ys_, zs, linewidths=1, alpha=0.5)
fig.colorbar(im, ax=ax)
ax.set_aspect('equal')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})
ax.plot_surface(xs_, ys_, zs, vmin=zs.min(), cmap='rainbow')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## Visualization of the electrostatic field

In [ ]:
Exs = np.zeros((N, N))
Eys = np.zeros((N, N))
Es = np.zeros((N, N))

for i in range(1, N-1):
    for j in range(1, N-1):
        Ex = -(phi[i+1, j]-phi[i-1, j])/(2.0*delta)
        Ey = -(phi[i, j+1]-phi[i, j-1])/(2.0*delta)
        Exs[i, j] = Ex
        Eys[i, j] = Ey
        Es[i, j] = np.linalg.norm((Ex, Ey))

In [ ]:
fig, ax = plt.subplots(figsize=None)
xs, ys = np.meshgrid(np.arange(N), np.arange(N))
zs = Es[xs, ys]
us = Exs[xs, ys]
vs = Eys[xs, ys]
xs_, ys_ = np.meshgrid(np.arange(N)*delta, np.arange(N)*delta)
im = ax.pcolormesh(xs_, ys_, zs, vmin=np.min(zs), vmax=np.max(zs), cmap='rainbow') # or jet
fig.colorbar(im, ax=ax)
#ax.streamplot(xs_, ys_, Exs, Eys, linewidth=1, cmap=plt.cm.inferno, density=2, arrowstyle='->', arrowsize=1.5)
#ax.contour(xs_, ys_, zs, linewidths=1, alpha=0.5)
if N <= 50:
    ax.quiver(xs_, ys_, us, vs, linewidth=1, minlength=3, cmap=plt.cm.inferno, alpha=0.5)
ax.set_aspect('equal')
plt.xlabel('x')
plt.xlabel('y')
plt.show()

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})
ax.plot_surface(xs_, ys_, zs, vmin=zs.min(), cmap='rainbow')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## Debug

In [ ]:
def dump_array(a: np.ndarray, tarnspose=False):
    np.set_printoptions(linewidth=100, formatter={'float': '{: 0.2f}'.format})
    vec = a.T if tarnspose else a
    print(vec)